# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [40]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from pprint import pprint

In [41]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [42]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [43]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [44]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [45]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [46]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [47]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [50]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    # print(response)
    result = response.choices[0].message.content
    return json.loads(result)
    return response

In [53]:
get_links("https://techcrunch.com")

{'links': [{'type': 'home page', 'url': 'https://techcrunch.com'},
  {'type': 'about page', 'url': 'https://techcrunch.com/about-techcrunch/'},
  {'type': 'careers page', 'url': 'https://www.crunchboard.com/'},
  {'type': 'contact page', 'url': 'https://techcrunch.com/contact-us/'},
  {'type': 'advertising page', 'url': 'https://techcrunch.com/advertise/'}]}

In [54]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/smolagents',
 '/microsoft/phi-4',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/DeepSeek-V3',
 '/black-forest-labs/FLUX.1-dev',
 '/NovaSky-AI/Sky-T1-32B-Preview',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/osanseviero/gemini-coder',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/reach-vb/2024-ai-timeline',
 '/spaces/fffiloni/LatentSync',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/DAMO-NLP-SG/multimodal_textbook',
 '/datasets/cfahlgren1/react-code-instructions',
 '/datasets/PowerInfer/QWQ-LONGCOT-500K',
 '/datasets/OpenLeecher/lmsys_chat_1m_clean',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '

In [55]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion page',
   'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [57]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [58]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
Use Ollama with GGUF Models from the HF Hub
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
microsoft/phi-4
Updated
4 days ago
•
42.8k
•
1.02k
hexgrad/Kokoro-82M
Updated
6 days ago
•
9.72k
•
746
deepseek-ai/DeepSeek-V3
Updated
13 days ago
•
112k
•
1.69k
black-forest-labs/FLUX.1-dev
Updated


In [61]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [62]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [63]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nUse Ollama with GGUF Models from the HF Hub\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nmicrosoft/phi-4\nUpdated\n4 days ago\n•\n42.8k\n•\n1.02k\nhexgrad/Kokoro-82M\nUpdated\n6 days ago\n•\n9.72k\n•\n747\ndeepseek-ai/DeepSeek-V3\nUpdated\n13 days ago\n•\n112k\n•\n1.69k\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16, 2024\n•\n1.25M\n•\n7.93k\nNovaSky-AI/Sky-T1-32B-Preview\nUpdated

In [66]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [67]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'company page', 'url': 'https://huggingface.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## Welcome to Hugging Face

At Hugging Face, we are dedicated to building the future of AI. We are more than just a company; we are a vibrant community that collaborates on cutting-edge models, extensive datasets, and innovative applications in the machine learning landscape.

---

## Our Platform

### Collaborate and Innovate
- **Models**: Access and share over **400k** models across various domains including text, audio, and images.
- **Datasets**: Browse and utilize more than **100k** datasets designed for diverse machine learning tasks.
- **Spaces**: Host and manage your own applications, making collaboration both smooth and scalable.

### Open Source Community
We pride ourselves on our open-source ethos, providing essential ML tooling that includes:
- **Transformers**: For state-of-the-art ML in PyTorch, TensorFlow, and JAX with 137k+ downloads.
- **Diffusers**: Premium models for audio and image generation.
- **Safetensors**: For secure storage of neural network weights.

---

## Our Customers

Over **50,000** organizations trust Hugging Face to power their AI initiatives, including industry leaders like:
- Microsoft
- Google
- Amazon
- Grammarly
- Intel

We serve a broad array of customers from non-profits to large enterprises, ensuring that everyone can leverage the power of machine learning.

---

## Company Culture

### A Supportive Environment
At Hugging Face, our culture is centered around inclusivity, collaboration, and innovation. We encourage our team members to share ideas, explore new technologies, and grow both personally and professionally.

### Community-driven Approach
We believe that the best products come from the community, which is why we actively seek feedback and contributions from our users. Our platform is built with collective knowledge and passion for AI at its core.

---

## Careers at Hugging Face

### Join Our Team
Become a part of our mission to bring AI to everyone. We are always on the lookout for talented individuals in various domains including engineering, research, and community engagement.

- **Inclusive Culture**: We value diverse perspectives, innovative thoughts, and the input of each team member.
- **Growth Opportunities**: We offer robust professional development options, mentorship programs, and career growth paths.

### Current Openings
Visit our **Jobs Page** to explore available positions and start your journey with Hugging Face. 

---

## Conclusion

Join the Hugging Face community to be at the forefront of AI innovation. Whether you’re a developer, an investor, or someone looking to start a fulfilling career, we invite you to be a part of a collective movement that is shaping the future of artificial intelligence.

---

### Connect with Us!

- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.com/invite/huggingface)

Together, let’s build the future of AI!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [68]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [69]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}]}


# Hugging Face Brochure

---

## About Us

**Hugging Face** is at the forefront of the artificial intelligence landscape, serving as a collaborative platform where the machine learning community can come together to create, discover, and innovate. With over **400,000 models**, **100,000 datasets**, and a rich array of cutting-edge applications, Hugging Face is building the future of AI by empowering developers and organizations to harness the power of machine learning.

---

## Our Offerings

- **Models**: Access a diverse library of state-of-the-art machine learning models, including those for text, images, videos, and audio.
  
- **Datasets**: Discover thousands of datasets for machine learning tasks, carefully curated to foster research and application development.
  
- **Spaces**: Deploy and showcase applications effortlessly with our user-friendly platform designed for collaboration.

- **Enterprise Solutions**: Benefit from advanced platform features that prioritize security and dedicated support, tailored for organizations of all sizes.

- **Compute Resources**: Deploy your models with optimized inference endpoints and scale seamlessly with paid compute solutions.

---

## Community & Culture

At Hugging Face, we pride ourselves on being the **AI community building the future**. We are committed to:
- **Open Source Collaboration**: Our core values are rooted in open-source principles, ensuring that all contributors can access and advance our tools and resources.
  
- **Continuous Learning**: We cultivate an environment that encourages learning and sharing ideas, making it easier for individuals and teams to innovate.

- **Supportive Atmosphere**: As part of our culture, we foster an inclusive workspace where every voice is valued and collaboration is prioritized.

---

## Who We Serve

We proudly partner with **more than 50,000 organizations**, including giants like:
- **Amazon Web Services**
- **Meta**
- **Google**
- **Microsoft**

Our clientele ranges from innovative startups to established enterprises, all leveraging our resources to enhance their AI and machine learning capabilities.

---

## Careers at Hugging Face

Join our growing team and be a part of groundbreaking AI developments! We are looking for passionate individuals who thrive in a dynamic, collaborative environment. Career opportunities range from engineering and product management to research and community engagement.

**Open Positions**: Check our [Jobs Page](https://huggingface.co/jobs) for the latest openings.

---

## Connect with Us

Stay updated and engage with our community:

- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)
- [Discord](https://discord.com/invite/huggingface)

---

Embrace the future of AI with Hugging Face – **where innovation meets collaboration**!

In [70]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

---

## About Us

**Hugging Face** is more than just a company; we are a thriving community dedicated to building the future of Artificial Intelligence (AI) and Machine Learning (ML). Our platform serves as a collaborative ecosystem for developers, researchers, and organizations to share, discover, and work on machine learning models, datasets, and applications seamlessly.

---

## Our Offerings

- **700K+ Models**: Browse a vast collection of machine learning models, including state-of-the-art solutions for various tasks.
- **150K+ Applications**: Discover innovative applications built on machine learning technology that advances industries and enhances daily life.
- **100K+ Datasets**: Access and contribute to our extensive library of datasets for diverse domains including text, image, and audio processing.

### Key Features
- **Collaboration Platform**: Host and collaborate on unlimited public models and datasets, empowering you to move faster with open-source tools.
- **Enterprise Solutions**: We cater to over 50,000 organizations, providing enterprise-grade security, access controls, and dedicated support.

---

## Our Customers

Hugging Face serves a wide range of organizations, including technology giants and non-profits:
- **Microsoft**
- **Google**
- **Amazon Web Services**
- **Meta**
- **Grammarly**
  
Our platform is trusted by enterprises, start-ups, and individual developers alike, giving them the tools to innovate in AI and ML.

---

## Company Culture

At **Hugging Face**, we believe in fostering an inclusive and innovative culture that inspires creativity and collaboration. We are dedicated to supporting the developer community through open-source initiatives and providing a space where ideas and projects can flourish. Our team embraces a growth mindset, allowing us to adapt and thrive in a rapidly evolving technological landscape.

### Core Values
- **Community Driven**: We prioritize the collective contributions of our users and developers, encouraging collaboration and knowledge sharing.
- **Innovation**: We champion continuous learning and improvement, striving to advance the field of machine learning responsibly and ethically.

---

## Careers at Hugging Face

Join us to be part of an exciting journey in shaping the future of AI. We offer dynamic job opportunities across various functions where you can thrive and make an impact.

### Why Work With Us?
- **Impactful Work**: Contribute to groundbreaking projects that influence various sectors and improve lives.
- **Collaborative Environment**: Work alongside passionate experts who are committed to sharing knowledge and driving innovation.
- **Growth Opportunities**: We support your career development and encourage you to reach your full potential.

*Explore open positions on our [Careers Page](https://huggingface.co/jobs)*

---

**Get In Touch!**

- **Website**: [Hugging Face](https://huggingface.co)
- **Social Media**: Follow us on GitHub, Twitter, LinkedIn, and Discord for the latest updates and community discussions. 

Join us in building the future of AI! 🚀

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>